In [0]:
!kill -9 -1

In [0]:
!ls

## Connect Drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import sys
sys.path.insert(0, 'drive/app')

In [0]:
!ls

In [0]:
import os 
directory=os.listdir("drive/app/wider face")
directory

In [0]:

os.chdir("drive/app")

In [0]:
!ls

In [0]:
os.chdir("/content/")

In [0]:
!ls

## Get Data


In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import os 
os.getcwd()

'/content'

In [3]:
!git clone https://github.com/omrylcn/python-widerface.git
  

Cloning into 'python-widerface'...
remote: Counting objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), done.


In [4]:
os.listdir("python-widerface")

['requirement.txt',
 '.git',
 'get_data.py',
 '.gitignore',
 'setup.py',
 'get_val_data.py',
 'README.md',
 'wider',
 'LICENSE']

In [5]:
os.chdir("python-widerface")
!ls

get_data.py	 LICENSE    requirement.txt  wider
get_val_data.py  README.md  setup.py


In [6]:
!python get_data.py
!python get_val_data.py

downloading the images from google drive...
downloading the bounding boxes annotations...
done
downloading the images from google drive...
done


In [7]:
!ls

data	     get_val_data.py  README.md        setup.py
get_data.py  LICENSE	      requirement.txt  wider


In [8]:
os.listdir("data")

['WIDER_val',
 'wider_face_split',
 'WIDER_train',
 'WIDER_train.zip',
 'wider_face_split.zip',
 'WIDER_val.zip']

In [9]:
os.getcwd()


'/content/python-widerface'

In [10]:
os.chdir("/content/")
!ls

python-widerface  sample_data


## Data to Tfrecord

In [11]:
!git clone https://github.com/omrylcn/widerface-to-tfrecord.git
!ls

Cloning into 'widerface-to-tfrecord'...
remote: Counting objects: 34, done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 34 (delta 2), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (34/34), done.
python-widerface  sample_data  widerface-to-tfrecord


In [12]:
os.listdir("widerface-to-tfrecord")

['wider_to_tfrecord.py',
 'utils',
 '.git',
 '.gitignore',
 'README.md',
 'config.py',
 'requirements.txt']

In [0]:
# function for copy  data to any folder 
import shutil

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

In [14]:


src="/content/python-widerface/data"

dst="/content/widerface-to-tfrecord"
copytree(src,dst)
os.listdir("widerface-to-tfrecord")

['WIDER_val',
 'wider_to_tfrecord.py',
 'utils',
 '.git',
 'wider_face_split',
 '.gitignore',
 'README.md',
 'WIDER_train',
 'WIDER_train.zip',
 'wider_face_split.zip',
 'config.py',
 'WIDER_val.zip',
 'requirements.txt']

In [0]:
os.makedirs("widerface-to-tfrecord/output") # output folder for tfrecord data

In [16]:
os.listdir("widerface-to-tfrecord")

['WIDER_val',
 'output',
 'wider_to_tfrecord.py',
 'utils',
 '.git',
 'wider_face_split',
 '.gitignore',
 'README.md',
 'WIDER_train',
 'WIDER_train.zip',
 'wider_face_split.zip',
 'config.py',
 'WIDER_val.zip',
 'requirements.txt']

In [17]:
os.listdir("widerface-to-tfrecord/wider_face_split")

['wider_face_train.mat',
 'wider_face_val.mat',
 'wider_face_test_filelist.txt',
 'readme.txt',
 'wider_face_val_bbx_gt.txt',
 'wider_face_test.mat',
 'wider_face_train_bbx_gt.txt']

In [18]:
!ls

python-widerface  sample_data  widerface-to-tfrecord


In [0]:
!python widerface-to-tfrecord/config.py

In [20]:
!cat widerface-to-tfrecord/config.py

# Training
TRAIN_WIDER_PATH = "/content/widerface-to-tfrecord/WIDER_train/"

# Validation
VAL_WIDER_PATH = "/content/widerface-to-tfrecord/WIDER_val/"

# Testing
TEST_WIDER_PATH = "/content/widerface-to-tfrecord/WIDER_test/"

# Ground Truth
GROUND_TRUTH_PATH = "/content/widerface-to-tfrecord/wider_face_split/"

# Output
OUTPUT_PATH = "/content/widerface-to-tfrecord/output/"


In [21]:
!python widerface-to-tfrecord/wider_to_tfrecord.py

Processing /content/widerface-to-tfrecord/WIDER_train/images
Correctly created record for 12880 images

Processing /content/widerface-to-tfrecord/WIDER_val/images
Correctly created record for 3226 images

Processing /content/widerface-to-tfrecord/WIDER_test/images
Correctly created record for 0 images



In [22]:
os.listdir("widerface-to-tfrecord/output")

['test.tfrecord', 'train.tfrecord', 'val.tfrecord']

In [0]:
os.remove("widerface-to-tfrecord/output/test.tfrecord")

In [24]:
os.listdir("/content/widerface-to-tfrecord/output")

['train.tfrecord', 'val.tfrecord']

In [25]:
!git clone https://github.com/omrylcn/colab.git

Cloning into 'colab'...
remote: Counting objects: 15, done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [26]:
os.listdir("colab")

['train.py',
 'ssdlite_mobilenet_v2_coco.config',
 'labelmap.pbtxt',
 '.git',
 'README.md']

In [0]:
os.makedirs("detection")

In [0]:
os.makedirs("detection/training")

In [29]:
shutil.move("colab/labelmap.pbtxt","detection/training")
shutil.move("colab/ssdlite_mobilenet_v2_coco.config","detection/training")
shutil.move("colab/train.py","detection")

'detection/train.py'

In [30]:
os.listdir("detection")

['train.py', 'training']

In [31]:
!ls

colab  detection  python-widerface  sample_data  widerface-to-tfrecord


In [32]:
# tfrecord file transport 
dst="/content/detection"

src="/content/widerface-to-tfrecord/output"
copytree(src,dst)
os.listdir("detection")

['train.py', 'training', 'train.tfrecord', 'val.tfrecord']

## Setup File

In [33]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install contextlib2
!pip install lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-tk is already the newest version (2.7.14-1).
The following additional packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6 libwebpmux3
  libxslt1.1 python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  liblcms2-utils python-genshi python-lxml-dbg python-lxml-doc python-pil-doc
  python-pil-dbg python-setuptools
The following NEW packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6 libwebpmux3
  libxslt1.1 protobuf-compiler python-bs4 python-chardet python-html5lib
  python-lxml python-olefile python-pil python-pkg-resources python-six
  python-webencodings
0 upgraded, 18 newly installed, 0 to remove and 0 not upgraded.
Need to get 3,751 kB of archives.
After this operation, 14.6 MB of additional disk space will be used.
Get:1 

In [34]:
!ls

colab  detection  python-widerface  sample_data  widerface-to-tfrecord


In [35]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Counting objects: 21583, done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 21583 (delta 0), reused 1 (delta 0), pack-reused 21575
Receiving objects: 100% (21583/21583), 558.38 MiB | 33.79 MiB/s, done.
Resolving deltas: 100% (12747/12747), done.
Checking out files: 100% (2670/2670), done.


In [36]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

Cloning into 'cocoapi'...
remote: Counting objects: 947, done.
remote: Total 947 (delta 0), reused 0 (delta 0), pack-reused 947
Receiving objects: 100% (947/947), 11.69 MiB | 22.71 MiB/s, done.
Resolving deltas: 100% (565/565), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -g -fdebug-prefix-map=/build/python3.6-sXpGnM/python3.6-3.6.3=. -specs=/usr/share/dpkg/no-pie-compile.specs -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99


In [0]:
os.chdir("/content/models/research")


In [38]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

env: PYTHONPATH=/content/models/research:/content/models/research/slim


In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [40]:
!python object_detection/builders/model_builder_test.py

..................
----------------------------------------------------------------------
Ran 18 tests in 0.137s

OK


### Test System

In [41]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3869799742972002546, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281989632
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9112135721130839312
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [42]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Transfer detection folder to models/research/object_detection folder

In [43]:
os.listdir("/content/models/research/object_detection")

['eval_util.py',
 'matchers',
 'CONTRIBUTING.md',
 'data',
 'test_data',
 'eval_util_test.py',
 '__init__.py',
 'export_tflite_ssd_graph.py',
 'test_ckpt',
 'export_tflite_ssd_graph_lib.py',
 'export_tflite_ssd_graph_lib_test.py',
 'legacy',
 'utils',
 'dockerfiles',
 'model_main.py',
 'inputs.py',
 'export_inference_graph.py',
 'models',
 'model_lib_test.py',
 'protos',
 'box_coders',
 '__pycache__',
 'predictors',
 'samples',
 'object_detection_tutorial.ipynb',
 'inference',
 'meta_architectures',
 'model_lib.py',
 'data_decoders',
 'exporter_test.py',
 'model_tpu_main.py',
 'builders',
 'core',
 'README.md',
 'test_images',
 'inputs_test.py',
 'dataset_tools',
 'g3doc',
 'exporter.py',
 'anchor_generators',
 'model_hparams.py',
 'metrics']

In [44]:
# "detection" folder transport to "object_detection" folder
src="/content/detection"

dst="/content/models/research/object_detection"
copytree(src,dst)
os.listdir("/content/models/research/object_detection")

['train.py',
 'eval_util.py',
 'matchers',
 'training',
 'CONTRIBUTING.md',
 'data',
 'test_data',
 'eval_util_test.py',
 '__init__.py',
 'export_tflite_ssd_graph.py',
 'test_ckpt',
 'export_tflite_ssd_graph_lib.py',
 'export_tflite_ssd_graph_lib_test.py',
 'legacy',
 'utils',
 'train.tfrecord',
 'dockerfiles',
 'model_main.py',
 'inputs.py',
 'export_inference_graph.py',
 'models',
 'model_lib_test.py',
 'protos',
 'box_coders',
 '__pycache__',
 'predictors',
 'samples',
 'object_detection_tutorial.ipynb',
 'inference',
 'meta_architectures',
 'model_lib.py',
 'data_decoders',
 'exporter_test.py',
 'model_tpu_main.py',
 'builders',
 'core',
 'README.md',
 'test_images',
 'inputs_test.py',
 'val.tfrecord',
 'dataset_tools',
 'g3doc',
 'exporter.py',
 'anchor_generators',
 'model_hparams.py',
 'metrics']

In [45]:
os.chdir("/content/models/research/object_detection")
!ls

anchor_generators		     matchers
box_coders			     meta_architectures
builders			     metrics
CONTRIBUTING.md			     model_hparams.py
core				     model_lib.py
data				     model_lib_test.py
data_decoders			     model_main.py
dataset_tools			     models
dockerfiles			     model_tpu_main.py
eval_util.py			     object_detection_tutorial.ipynb
eval_util_test.py		     predictors
exporter.py			     protos
exporter_test.py		     __pycache__
export_inference_graph.py	     README.md
export_tflite_ssd_graph_lib.py	     samples
export_tflite_ssd_graph_lib_test.py  test_ckpt
export_tflite_ssd_graph.py	     test_data
g3doc				     test_images
inference			     training
__init__.py			     train.py
inputs.py			     train.tfrecord
inputs_test.py			     utils
legacy				     val.tfrecord


In [46]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!tar -xvf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

--2018-09-01 20:46:41--  http://storage.googleapis.com/download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51025348 (49M) [application/x-tar]
Saving to: ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz’

ssdlite_mobilenet_v 100%[===================>]  48.66M   189MB/s    in 0.3s    

2018-09-01 20:46:41 (189 MB/s) - ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz’ saved [51025348/51025348]

ssdlite_mobilenet_v2_coco_2018_05_09/checkpoint
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.data-00000-of-00001
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.meta
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.index
ssdlite_mobilenet_v2_coco_2018_05_09/saved_model/saved_model.pb
ssdlite_mobilenet_v2_coco_2018_05_09

In [47]:
os.listdir()

['train.py',
 'eval_util.py',
 'matchers',
 'training',
 'CONTRIBUTING.md',
 'ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz',
 'data',
 'test_data',
 'eval_util_test.py',
 '__init__.py',
 'export_tflite_ssd_graph.py',
 'test_ckpt',
 'export_tflite_ssd_graph_lib.py',
 'export_tflite_ssd_graph_lib_test.py',
 'legacy',
 'utils',
 'train.tfrecord',
 'dockerfiles',
 'model_main.py',
 'inputs.py',
 'export_inference_graph.py',
 'models',
 'model_lib_test.py',
 'protos',
 'box_coders',
 '__pycache__',
 'predictors',
 'samples',
 'object_detection_tutorial.ipynb',
 'inference',
 'meta_architectures',
 'model_lib.py',
 'data_decoders',
 'exporter_test.py',
 'model_tpu_main.py',
 'builders',
 'core',
 'README.md',
 'test_images',
 'inputs_test.py',
 'val.tfrecord',
 'dataset_tools',
 'g3doc',
 'exporter.py',
 'anchor_generators',
 'model_hparams.py',
 'ssdlite_mobilenet_v2_coco_2018_05_09',
 'metrics']

In [48]:
os.listdir("training")

['ssdlite_mobilenet_v2_coco.config', 'labelmap.pbtxt']

In [0]:
os.getcwd()

In [49]:
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssdlite_mobilenet_v2_coco.config

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Use the `axis` argument instead
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2018-09-01 20:47:41.002069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-09-01 20:47:41.002494: I tensorflow

In [0]:
!cat /proc/meminfo